In [4]:
# This program scraps a news about specific topic from newsapi.org API.
# To change topic (or select topic), change q = '' part
# To get specific news domain, use domains = '<news domain url>&'
# Use page=#& to get more results

# Author, title, description, url, content, publishing date, source ID 

# Import required modules
import requests
import numpy as np
import pandas as pd
import csv


def news_api(topic, number, source, date, file_name):
    # List creation
    source_id = []
    author = []
    title = []
    description = []
    url = []
    content = []
    pub_date = []


    # Read 100 news articles about coronavirus (20 articles per each page, 5 pages) and parse each data into corresponding lists
    for i in range(1, int((number/20)+1)):

        # News from CNN
        api = ('https://newsapi.org/v2/everything?'
                f'domains={source}&'
                f'q={topic}&'
               f'page={i}&'
               f'from={date}&'
               'sortBy=popularity&'
               'apiKey=ddb22f3ef2b54abe8af228db83421424')

        data = requests.get(api)

        # Testing for data
        #print(api.json()['articles'])
    
        for elements in data.json()['articles']:
            source_id.append(elements['source']['id'])
            author.append(elements['author'])
            title.append(elements['title'])
            description.append(elements['description'])
            url.append(elements['url'])
            content.append(elements['content'])
            pub_date.append(elements['publishedAt'])

            # *For CNN Only: check if the content is video and data does not related to the article
            #if (elements['content'] == 'None' or elements['content'] == "Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds."):
            #    pass
                #print(elements['description'])
            #else:
                #print (elements['content'])
            #    pass

    # Test for parsed data    
    #print(author)

    # Series Creation
    source_id_ser = pd.Series(source_id)
    author_ser = pd.Series(author)
    title_ser = pd.Series(title)
    des_ser = pd.Series(description)
    url_ser = pd.Series(url)
    content_ser = pd.Series(content)
    date_ser = pd.Series(pub_date)

    # Concatinate data of articles in one Series
    News_data = pd.concat([source_id_ser, author_ser, title_ser, des_ser, url_ser, content_ser, date_ser], keys = ['source_id','author','title','description','url','content','pub_date'], axis = 1)
    #print(News_data)

    new_header = ['source_id', 'author', 'title', 'description', 'url', 'content', 'published_date']

    # Write csv file with raw data of news article
    News_data.to_csv(f'{file_name}', header = new_header)
    
    return News_data

news = news_api('coronavirus', 100, 'cnn.com', '2020-01-27', './datasets/ref_news.csv')
text = news_api('coronavirus', 100, 'theverge.com, foxnews.com', '2020-01-27', './datasets/test_text.csv')

# Checking the data
print(news.head(2))
print(text.head(2))

# Just saving some other methods
#data = pd.read_csv(f'./datasets/liar_dataset/new_dataset.csv', delimiter=',')
#for elements in range(len(data)):
#    content = data.loc[elements, 'content']
#    description = data.loc[elements, 'description']

  source_id                      author  \
0       cnn  Shannon Liao, CNN Business   
1       cnn       Michael Nedelman, CNN   

                                               title  \
0  Apple temporarily closes all 42 stores in Chin...   
1  New report on first US case of the virus detai...   

                                         description  \
0  Apple is temporarily closing all of its stores...   
1  Doctors have shared new details about the firs...   

                                                 url content  \
0  https://www.cnn.com/2020/02/01/tech/apple-chin...    None   
1  https://www.cnn.com/2020/01/31/health/washingt...    None   

               pub_date  
0  2020-02-01T20:52:44Z  
1  2020-01-31T22:47:16Z  
   source_id           author  \
0  the-verge   Nicole Wetsman   
1  the-verge  Eleanor Cummins   

                                               title  \
0  Universal coronavirus treatments could help tr...   
1  The new coronavirus is not an excuse to be rac

In [10]:
'''
This program is only for feature extraction from given data.

At the beginning, the program grab the news contents (if content is null, use description. If both are
null, drop that news.) from the given dataset and extract the features from the news contents.
Then, it saves extracted data and features in a file.

Features used in this program are:
1. Number of Characters
2. Number of Words
3. Number of Verbs
4. Number of Nouns
5. Number of Sentence
6. Average Number of Words per Sentence
7. Average Number of Characters in Words
8. Number of Question Marks
9. Percentage of Subjective Verbs
10. Percentage of Passive Voice
11. Percentage of Positive Words
12. Percentage of Negative Words
13. Lexical Diversity: Unique Words or Terms
14. Typographical Error Ration: Misspelled Words
15. Causation Terms

How to use: First, place positive-words.txt and negative-words.txt files in word_sentiment folder and put
            .csv dataset in liar_dataset folder in datasets folder and put the name of the .csv file
            in argument space of setup_data function. Next, use feature_extraction function with the result of
            setup_data function to extract the features and save the data in the desired file.

Date: Jan 28th, 2020
Programmed by: Sasung Kim
'''

# Importing Required modules

import numpy as np
import pandas as pd
import csv

from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re
from collections import Counter

#########################################################################################################
# Create the lists of the words that used for feature extraction

positive_list = list()
negative_list = list()

# Gather the positive and negative word lists from the text file
with open('./word_sentiment/positive-words.txt') as p:
    for line in p:
        val = line.split()
        positive_list.append(val)

with open('./word_sentiment/negative-words.txt') as n:
    for line in n:
        val = line.split()
        negative_list.append(val)

subjective_list = list(['am ', 'are ', 'is ', 'was ', 'were ', 'be ', 'been '])
causation_list = list(['led to ', 'because ', 'cause ', 'reason ', 'explanation ', 'so '])

exclusive_list = list(['except ', 'else ', 'besides ', 'without ', 'exclude ', 'other than '])
generalizing_list = list(['all ', 'none ', 'most ', 'many ', 'always ', 'everyone ','never ',
                          'some ','usually ','few ','seldom ','generally ','general ','overall '])
pronoun_1st_list = list(['I ','we '])
pronoun_2nd3rd_list = list(['you','your','yours','he','she','it','him','her','his','her','its','hers','They','them','theirs','their'])

SPronoun_list = list(['I', 'mine','my','me'])
GPronoun_list = list(['we','ours','our','us'])

#########################################################################################################

# From the dataset, isolate the required data (statement and label in this program) and return as pandas Series
# If the user needs the required data as a file, uncomment def with new_file as argument and comment the other one

#def setup_data(file_name, new_file):
def setup_data(file_name):
    data = pd.read_csv(f'{file_name}', delimiter=',')

    # Check for the raw data from original dataset
    # print (list(data.columns.values))
    # print(data.tail(1))

    news_contents = []
    
    for elements in range(len(data)):
        description = data.loc[elements, 'description']
        
        # uncomment and indent the news_contents.append(description) line if both contents and descriptions are used instead of  description only
        
        #content = data.loc[elements, 'content']
        #if not pd.isna(content) and content is "Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds.":
        #    news_contents.append(content)
        #else:
        news_contents.append(description)
    
    # drop the null values (if both content and description are null)
    news_contents = [x for x in news_contents if x == x]
    
    # Check if the required data are correctly input
    #print(news_contents)
    
    return news_contents


#########################################################################################################

# Not required for right now, but may used in later for more efficient tockenization and feature extraction
# vectorizer = CountVectorizer()
# train_count = vectorizer.fit_transform(data["statement"].values)

# Check for vectorizer
# print(train_count.shape)
# print(vectorizer.vocabulary_)
# print(vectorizer.get_feature_names())

# Input: String ('str')
# Description: Tockenize and tag input with nltk universal tag
# Return: tags of each tockens - String ('tagged')
# Tockenize and tag input String with nltk universal tag and return the tags for each words in the String

def tagging_univ(str):
    text = nltk.word_tokenize(str)
    tagged = nltk.pos_tag(text, tagset='universal')
    return tagged

# Input: String('str')
# Description: Tockenize and tag input with nltk non-universal tag
# Return: tags of each tockens - String ('tagged')

def tagging_nuniv(str):
    text = nltk.word_tokenize(str)
    tagged = nltk.pos_tag(text)
    return tagged

# Input: String('str')
# Description: Count the number of characters in input
# Return: Character count - int ('count')

def count_char(str):
    no_space = str.replace(" ", "")
    count = len(no_space)
    return count

# Input: String('str')
# Description: Count the number of words in input
# Return: Word count - int ('count')

def count_word(str):
    count = len(str.split())
    return count

# Input: String ('tags'), int(verb_count)
# Description: Count the number of verbs in input
# Return: Verb count - int ('verb_count')

def count_verb(tags, verb_count):
    if (tags == 'VERB'):
        verb_count += 1
    return verb_count

# Input: String('tags'), int('noun_count')
# Description: Count the number of nouns in input
# Return: Noun count - int ('noun_count')

def count_noun(tags, noun_count):
    if (tags == 'NOUN'):
        noun_count += 1
    return noun_count

# Input: String ('str')
# Description: Count the number of sentences by counting number of period(.)
# Return: Sentence count - int ('sentence')

def count_sent(str):
    sentence = len(str.split('.'))
    return sentence

# Input: String ('states')
# Description: Count the number of characters in each word in input and average the number of characters per word
# Return: Average number of characters: float ('avg')

def count_char_per_word(states):
    word = []
    word.append(states.split())
    char_per_word = list()
    for elements in word:
        for char in elements:
            c_in_w_count = len(char)
            char_per_word.append(c_in_w_count)
    # char_per_word_list.append()
    avg = sum(char_per_word) / len(char_per_word)
    char_per_word.clear()
    return avg

# Input: String ('tagged'), list ('list'), int ('int')
# Description: Count the number of common words between input String and input list
# Return: Count of common words - int ('int')
# Count and return the number of common words between input String ('tagged') and input list

def check_common(tagged, list, int):
    for elements in list:
        if tagged in elements:
            int += 1
            return int
    return int

# Input: list ('subjective_list'), int ('verb_count'), String ('states')
# Description: Count the number of subjective words and avarage it from number of verbs in input
# Return: Average of subjective words - float ('percent_sub')

def count_sub(subjective_list, verb_count, states):
    percent_sub = 0
    for ele in subjective_list:
        sub_count = states.count(ele)
    if (verb_count > 0):
        percent_sub = sub_count / verb_count * 100
    return percent_sub

# Input: String ('word'), String ('tag'), list ('subjective_list'), int ('sent_count')
# Description: Count the number of passive voice and avarge it from number of sentence (sent_count)
# Return: Average of passive voiced sentences - float ('result')

def count_passive(word, tag, subjective_list, sent_count):
    percent_sub = 0
    counter = 0
    for ele in subjective_list:
        if (word.count(ele) > 0 and tag == "VBN"):
            counter += 1
    result = counter / sent_count * 100
    return result

# Input: String ('states')
# Description: Count the words that introduced only once in input
# Return: Count of unique words - int ('unique_count')

def count_unique(states):
    words = states.split(' ')
    c = Counter(words)
    unique = [w for w in words if c[w] == 1]
    unique_counter = len(unique)
    return unique_counter

# Input: String ('tag'), int ('typo_count')
# Description: Count the number of words with nltk universal tag is "X" (misspelled)
# Return: Count of misspelled words - int ('typo_count')

def count_typo(tag, typo_count):
    if (tag == "X"):
        typo_count += 1
    return typo_count

# Input: list ('causation_list'), String ('states')
# Description: Count the number of common words in list input and String input
# Return: Count of common words - int ('cause_count')

def count_cause(causation_list, states):
    for ele in causation_list:
        cause_count = states.count(ele)
    return cause_count

# Calvin

def count_num(tags, num_count):
    if(tags == 'NUM'):
        num_count += 1
    return num_count

def avg_char(states):
    word.append(states.split())
    char_per_word = list()
    for elements in word:
        for char in elements:
            c_in_w_count = len(char)
            char_per_word.append(c_in_w_count)
    word.clear()
    avg = sum(char_per_word)/len(char_per_word)
    char_per_word.clear()
    return avg

def avg_punc(tags, punc_count):
    if(tags == '.'):
        punc_count += 1
    return punc_count

def count_gene(generalizing_list, states):
    for ele in generalizing_list:
        gene_count = states.count(ele)
    return gene_count

def count_exclu(exclusive_list, states):
    for ele in exclusive_list:
        exclu_count = states.count(ele)
    return exclu_count

def count_pron_2nd3rd(pronoun_1st_list, states):
    for ele in pronoun_1st_list:
        pron_2nd3rd_count = states.count(ele)
    return pron_2nd3rd_count

def count_pron_1st(pronoun_1st_list, states):
    for ele in pronoun_1st_list:
        pron_1st_count = states.count(ele)
    return pron_1st_count

# Andrew

#self pronoun 1st person
def count_pronself(SPronoun_list, states):
    for ele in SPronoun_list:
        SelfP = states.count(ele)
        return SelfP
    
#Group ref 1st person
def count_prongroup(GPronoun_list, states):
    for ele in GPronoun_list:
        GP = states.count(ele)
        return GP
    
# 2nd and 3rd person pronouns
def count_pronother(Pronoun_list, states):
     for ele in Pronoun_list:
         othernouns = states.count(ele)
         return othernouns


# Input: panda Series ('data'), String ('save_file_name')
# Description: First, this function creates lists for each features and extract the features using statements in dataset and
#              above functions. Next, it creates a large pandas Series that consist of news contents, labels and extracted
#              features. Finally, it save the final pandas Series in a file to make easier to examine the result (do not need
#              to rerun the program or change the code to check raw data)
# Return: Pandas Series consist of news contents, labels and extracted features count - pandas Series ('new')

def feature_extract(data, save_file_name):
    # define the news contents and labels from the dataset
    state = data

    # create lists for storing the counters
    char_count_list = list()
    word_count_list = list()
    verb_count_list = list()
    noun_count_list = list()
    sent_count_list = list()
    words_per_sent_list = list()
    char_per_word_list = list()
    quest_count_list = list()
    sub_count_list = list()
    pass_count_list = list()
    pos_count_list = list()
    neg_count_list = list()
    unique_count_list = list()
    typo_count_list = list()
    cause_count_list = list()
    
    # Calvin
    gene_count_list = list()
    num_count_list = list()
    pron_1st_count_list = list()
    pron_2nd3rd_count_list = list()
    exclu_count_list = list()
    
    # Andrew
    exclam_list = list()
    lex_list = list()
    singlulars_list = list()
    group_list = list()
    other_list = list()


    word = list()

    #print(type(state))
    # loop for checking each new contents in dataset
    for states in state:

        #print(states)
        # reset the counters for each news contents
        w_in_s_count = 0
        c_in_w_count = 0
        verb_count = 0
        noun_count = 0
        pos_count = 0
        neg_count = 0
        percent_pos = 0
        percent_neg = 0
        unique_count = 0
        sent_counts = 0
        typo_count = 0
        num= 0
        

        # Tockenization and tagging with nltk universal and non-universal tag systems (tagged = universal, tagged_nu = non-universal)
        tagged = tagging_univ(states)
        tagged_nu = tagging_nuniv(states)

        # Check the tags of each news contents
        # print(tagged)
        # print(tagged_nu)

        # Extract the features and append the results in the list. Commented lines with print() functions are for testing

        # 1. Number of Characters
        char_count = count_char(states)
        char_count_list.append(char_count)

        # 2. Nubmer of Words
        word_count = count_word(states)
        word_count_list.append(word_count)

        # 3. Number of Verbs
        for tag in tagged:
            verb_count = count_verb(tag[1], verb_count)
        verb_count_list.append(verb_count)

        # 4. Number of Nouns
        for tag in tagged:
            noun_count = count_noun(tag[1], noun_count)
        noun_count_list.append(noun_count)
        # print(noun_count)

        # 5. Number of Sentence
        sent_count = count_sent(states)
        sent_count_list.append(sent_count)
        # print(sent_count)

        # 6. Average number of words per sentence
        sent = [len(l.split()) for l in re.split(r'[?!.]', states) if l.strip()]
        w_in_s_count = (sum(sent) / len(sent))
        words_per_sent_list.append(w_in_s_count)
        # print(w_in_s_count)

        # 7. Average number of characters per word
        c_in_w_count = count_char_per_word(states)
        char_per_word_list.append(c_in_w_count)
        # print(c_in_w_count)

        # 8. Number of question marks
        quest_count = states.count("?")
        quest_count_list.append(quest_count)

        # 9. Percentage of subjective verbs - am/are/is/etc
        percent_sub = count_sub(subjective_list, verb_count, states)
        sub_count_list.append(percent_sub)

        # 10. Percentage of passive voice - am/are/is && past participate
        for tag in tagged_nu:
            passive_percent = count_passive(tag[0], tag[1], subjective_list, sent_count)
        pass_count_list.append(passive_percent)

        # 11. Percentage of positive words
        for tag in tagged:
            pos_count = check_common(tag[0], positive_list, pos_count)
            percent_pos = pos_count / word_count * 100
        pos_count_list.append(percent_pos)

        # 12. Percentage of negative words
        for tag in tagged:
            neg_count = check_common(tag[0], negative_list, neg_count)
            percent_neg = neg_count / word_count * 100
        neg_count_list.append(percent_neg)

        # 13. Lexical diversity: unique words or terms
        unique_count = count_unique(states)
        unique_count_list.append(unique_count)

        # 14. Typographical error ratio: misspelled words
        for tag in tagged:
            typo_count = count_typo(tag[1], typo_count)
        typo_count_list.append(typo_count)

        # 15. Causation terms
        cause_count = count_cause(causation_list, states)
        cause_count_list.append(cause_count)

        # 16. Percentage of generalizing terms
        gene_count = count_gene(generalizing_list, states)
        gene_count_list.append(gene_count)
        
        # 17. Percentage of numbers and quantifiers
        for tag in tagged:
            num = count_num(tag[1], num)
        num_count_list.append(num)

        # 18. 1st person pronouns
        pron_1st_count = count_pron_1st(pronoun_1st_list, states)
        pron_1st_count_list.append(pron_1st_count)
   
        # 19. 2nd and 3rd person pronouns
        pron_count = count_pron_2nd3rd(pronoun_2nd3rd_list, states)
        pron_2nd3rd_count_list.append(pron_count)

        # 20. Exclusive terms
        exclu_count = count_exclu(exclusive_list, states)
        exclu_count_list.append(exclu_count)
        
        # 21. # of exclamation marks
        exclam_count = states.count("!")
        exclam_list.append(exclam_count)
        
        # 22. lexical 
        unique_count = count_unique(states)
        lex_list.append(unique_count)
        
        # 23. singlular pronouns (1st person)
        SelfP = count_pronself(SPronoun_list, states)
        singlulars_list.append(SelfP)
        
        # 24. Group ref pronouns (1st person)
        group = count_prongroup(GPronoun_list, states)
        group_list.append(group)
        
        # 25. 2nd 3rd pronouns
        other = count_pronother(pronoun_2nd3rd_list, states)
        other_list.append(other)
        
    # Convert lists into pandas Series
    statement = pd.Series(state)
    first = pd.Series(char_count_list)
    second = pd.Series(word_count_list)
    third = pd.Series(verb_count_list)
    fourth = pd.Series(noun_count_list)
    fifth = pd.Series(sent_count_list)
    sixth = pd.Series(words_per_sent_list)
    seventh = pd.Series(char_per_word_list)
    eighth = pd.Series(quest_count_list)
    nineth = pd.Series(sub_count_list)
    tenth = pd.Series(pass_count_list)
    eleventh = pd.Series(pos_count_list)
    twelveth = pd.Series(neg_count_list)
    thirteenth = pd.Series(unique_count_list)
    fourteenth = pd.Series(typo_count_list)
    fifteenth = pd.Series(cause_count_list)
    
    
    sixteenth = pd.Series(gene_count_list)
    seventeenth = pd.Series(num_count_list)
    eighteenth = pd.Series(pron_1st_count_list)
    nineteenth = pd.Series(pron_2nd3rd_count_list)
    twentieth = pd.Series(exclu_count_list)
    twenty_first = pd.Series(exclam_list)
    twenty_second = pd.Series(lex_list)
    twenty_third = pd.Series(singlulars_list)
    twenty_fourth = pd.Series(group_list)
    twenty_fifth = pd.Series(other_list)

    # Concatenate all feature Series, news contents and labels into one pandas Series
    new = pd.concat([statement, first, second, third, fourth, fifth, sixth, seventh, eighth, nineth, tenth,
                     eleventh, twelveth, thirteenth, fourteenth, fifteenth, sixteenth, seventeenth, eighteenth,
                    nineteenth, twentieth, twenty_first, twenty_second, twenty_third, twenty_fourth, twenty_fifth], axis=1)

    # Test for the final Series
    # print(new.head(3))

    
    # 16. Percentage of generalizing terms
 # 17. Percentage of numbers and quantifiers
# 18. 1st person pronouns
# 19. 2nd and 3rd person pronouns
# 20. Exclusive terms
# 21. # of exclamation marks
# 22. lexical
 # 23. singlular pronouns (1st person)
# 24. Group ref pronouns (1st person)
# 25. 2nd 3rd pronouns
    
    # Name the header of the data
    new_header = ['Statement', '# of Characters', '# of Words', '# of Verbs', '# of Noun', '# of Sentence',
                  'Average # of Words per Sentence', 'Average # of Characters per Words', '# of Question Marks',
                  '% of Subjective Verbs', '% of Passive Voice', '% of Positive Words', '% of Negative Words',
                  '# of Unique Wrods/Terms', '# of Misspelled Words', '# of Causation Terms', "% of generalizing terms",
                 "% of # and quantifiers", "1st person pronouns", "2nd and 3rd person pronouns", "Exclusive term",
                 "# of exclamation marks", "Lexical", "Singular pronouns(1st person)", "Group ref pronouns(1st person)",
                 "2nd 3rd pronouns"]

    # Save the data in a file

    #with open(save_file_name, 'w') as f:
    #    writer = csv.DictWriter(f, fieldnames=new_header)
    #    writer.writeheader()

    new.to_csv(save_file_name, header=new_header)
    
    return new


########################################################################################################################

# Data Extraction
# From the dataset file, gather the news contents and labels only

text_data = setup_data('./datasets/test_text.csv')
news_data = setup_data('./datasets/ref_news.csv')

# Check for shape and contents of isolated data
# print(train_data.shape)
# print(test_data.shape)
# print (valid_data.shape)
# print (train_data.head(5))
# print (test_data.head(5))
# print (valid_data.head(5))

# Feature Extraction

text_data_FE = feature_extract(text_data, './text_FE.csv')
news_data_FE = feature_extract(news_data, './news_FE.csv')

# Checking data

print(text_data_FE.head(5))
print(news_data_FE.head(5))


                                                  0    1   2   3   4   5   \
0  Coronaviruses caused SARS, MERS, and the curre...  150  28   7   9   3   
1  The coronavirus — which originated in Wuhan, C...  220  42   6  15   2   
2    The State Department issued the travel advisory   41   7   1   4   1   
3  Any foreign national who has traveled within C...  215  46  10  14   2   
4  The World Health Organization declared a Publi...  218  43   7  17   3   

          6         7   8    9   ...  16  17  18  19  20  21  22  23  24  25  
0  14.000000  5.357143   0  0.0  ...   0   0   0   0   0   0  21   0   0   0  
1  21.000000  5.214286   0  0.0  ...   0   2   0   0   0   0  32   0   0   0  
2   7.000000  5.857143   0  0.0  ...   0   0   0   0   0   0   7   0   0   0  
3  23.000000  4.673913   0  0.0  ...   0   1   0   0   0   0  37   0   1   0  
4  14.333333  5.069767   0  0.0  ...   0   2   0   0   0   0  28   2   0   0  

[5 rows x 26 columns]
                                        

In [1]:
"""
This program extract the multi-sourced features from two datasets (one is unreliable dataset that text that is evaluated, one for reliable dataset as reference news of evaluation).

Restrictions: Two provided datasets must have the same number of data and have same feature and name. For better use, it is recommended to use the news/texts that have same topics.

How to use: Place the datasets that consists of statements and feature values in a folder and put the name or path of the datasets in news1 and news2 arguments of multi_source_FE() definition.
            news1 should be the dataset of referencing articles, and news2 should be the dataset of testing texts

Date: Feb 2, 2020
Programmed by: Sasung Kim

"""

# Import Required Modules
import numpy as np
import pandas as pd
import csv
from statistics import mean

# Import Required Dictionaries for Feature Extraction

positive_list = list()
negative_list = list()

# Gather the positive and negative word lists from the text file
with open('./word_sentiment/positive-words.txt') as p:
    for line in p:
        val = line.split()
        positive_list.append(val)

with open('./word_sentiment/negative-words.txt') as n:
    for line in n:
        val = line.split()
        negative_list.append(val)

subjective_list = list(['am ', 'are ', 'is ', 'was ', 'were ', 'be ', 'been '])
causation_list = list(['led to ', 'because ', 'cause ', 'reason ', 'explanation ', 'so '])

# Define the definitions for extracting features

# Input: fileName (String)
# Description: Read the .csv file and return the data as list
# Return: data (list)

def data_read(fileName):
    with open (fileName, encoding = 'utf-8') as f:
        reader = csv.reader(f)
        first_row = next(reader, None)
        data = list(reader)
        return data, first_row

def header_extension(data_list, startColumn, extension, new_header):
    for i in range(startColumn, len(data_list)):
        new_header.append(f'{data_list[i]} - {extension}')
    return new_header
    
    
# Subtract each features for each data in the same row (one-to-one) and save the data in total_list. List contains
# the extracted feature values related to texts. Ex. total_list[0] = all feature values of first statement
# To use absolute value, uncomment first feature and comment second feature.

def one_to_one_dif(text, ref, startFColumn, feature_num):
    feature_list = []
    for i in range (startFColumn, feature_num):
        try:
            #feature = (abs(float(data2[i]) - float(data1[i])))
            feature = (float(text[i]) - float(ref[i]))
        except:
            feature = (float(text[i]))
                
        feature_list.append(round(feature, 2))
    return feature_list
    
    
def one_to_many_dif(ref, text, feature_num, feature_pos):
    avg_list = [];
    for i in range(len(ref)):
        feature_val = float(text[feature_pos]) - float(ref[i][feature_pos])
        avg_list.append(feature_val)
    avg = round(mean(avg_list), 2)
    return avg
    
    
# Input: news1 (String), news2 (String), outFile (String), startFColumn (integer)
# Description: Read the data from news1 (reliable) and news2 (unreliable), and put the data in two lists. Then, subtract the news1 values in each column from news2 values.
#              The subtraction starts from startFColumn until the end of the .csv file. Next, the results will be saved in the file with given name or path from user (outFile)
#
# Return: New DataFrame ('MS_features')

def multi_source_FE(news1, news2, outFile, startFColumn):
    
    # Reading data from .csv files
    data1 = data_read(f'./{news1}')
    data2 = data_read(f'./{news2}')
    
    # Saving original headers in header
    header = data1[1]
    feature_num = len(header)
    
    # Create header list with modified header names. Can be changed later with different method of multi-sourcing by modifying '- sub'.
    header_edit = ['text', 'news']
    header_edit = header_extension(header, 2, 'sub', header_edit)
    header_edit = header_extension(header, 2, 'avg sub', header_edit)
    
    #for i in range(startFColumn, len(data1[1])):
     #   header.append(f'{data1[1][i]} - sub')
      #  header.append(f'{data1[1][i]} - avg sub')
    
    # Assign data of the testing text and referencing text in data1 and data2
    data1 = data1[0]
    data2 = data2[0]
    
    # Create local lists, dataframe that are used later in this definition.
    text = []
    ref = []
    total_list = []
    new_total_list = []
    df = pd.DataFrame()
    
    # Loop through the data for subtraction
    for i in range(len(data2)):
        
        # Statement of each data (text - text trying to verify, ref - reliable articles to reference for text comparison)
        try:
            ref.append(data1[i][1])
            text.append(data2[i][1])
        except:
            pass
        
        #print(one_to_one_dif(data1[i], data2[i], startFColumn, feature_num))
        try:
            total_list.append(one_to_one_dif(data1[i], data2[i], startFColumn, feature_num))
            for j in range(startFColumn, feature_num):
                total_list[i].append(one_to_many_dif(data1, data2[i], feature_num, j))
        except:
            pass
        
    
    # Reshape the lists in terms of features. Ex. new_total_list[0] = all values of first feature
    for i in range(len(total_list)):
        new_list = []
        for element in (total_list):
            try:
                new_list.append(element[i])
            except:
                pass
        new_total_list.append(new_list)
       
    # Creating dataframe with multi-sourced features
    df['TEXT'] = text
    df['REFERENCE'] = ref
    for i in range(2, len(header_edit)):
        df[header_edit[i]] = new_total_list[i-2]
    
    # Saving data into file
    df.to_csv(f'./{outFile}')
    return df

# Initiate the code and check for the dataframe
MSFE = multi_source_FE('news_FE.csv', 'text_FE.csv', 'OtoO_FE.csv', 2)
print(MSFE.head(10))


                                                TEXT  \
0  Coronaviruses caused SARS, MERS, and the curre...   
1  The coronavirus — which originated in Wuhan, C...   
2    The State Department issued the travel advisory   
3  Any foreign national who has traveled within C...   
4  The World Health Organization declared a Publi...   
5  The Centers for Disease Control and Prevention...   
6  Coronavirus outbreak: What to do if you think ...   
7  Foxconn has stopped “almost all” of its produc...   
8  American Airlines announced that it’s suspendi...   
9  Mark Zuckerberg says people need to understand...   

                                           REFERENCE  # of Characters - sub  \
0  Apple is temporarily closing all of its stores...                  -72.0   
1  Doctors have shared new details about the firs...                  -85.0   
2  The Wuhan coronavirus outbreak is not a pandem...                  106.0   
3  As the coronavirus outbreak grows after killin...               

In [ ]:
"""
This program extract the multi-sourced features from two datasets (one is unreliable dataset that text that is evaluated, one for reliable dataset as reference news of evaluation).

Restrictions: Two provided datasets must have the same number of data and have same feature and name. For better use, it is recommended to use the news/texts that have same topics.

How to use: Place the datasets that consists of statements and feature values in a folder and put the name or path of the datasets in news1 and news2 arguments of multi_source_FE() definition.
            news1 should be the dataset of referencing articles, and news2 should be the dataset of testing texts

Date: Feb 2, 2020
Programmed by: Sasung Kim

"""

# Import Required Modules
import numpy as np
import pandas as pd
import csv
from statistics import mean

# Import Required Dictionaries for Feature Extraction

positive_list = list()
negative_list = list()

# Gather the positive and negative word lists from the text file
with open('./word_sentiment/positive-words.txt') as p:
    for line in p:
        val = line.split()
        positive_list.append(val)

with open('./word_sentiment/negative-words.txt') as n:
    for line in n:
        val = line.split()
        negative_list.append(val)

subjective_list = list(['am ', 'are ', 'is ', 'was ', 'were ', 'be ', 'been '])
causation_list = list(['led to ', 'because ', 'cause ', 'reason ', 'explanation ', 'so '])

# Define the definitions for extracting features

# Input: fileName (String)
# Description: Read the .csv file and return the data as list
# Return: data (list)

def data_read(fileName):
    with open (fileName, encoding = 'utf-8') as f:
        reader = csv.reader(f)
        first_row = next(reader, None)
        data = list(reader)
        return data, first_row

def header_extension(data_list, startColumn, extension, new_header):
    for i in range(startColumn, len(data_list)):
        new_header.append(f'{data_list[i]} - {extension}')
    return new_header
    
    
# Subtract each features for each data in the same row (one-to-one) and save the data in total_list. List contains
# the extracted feature values related to texts. Ex. total_list[0] = all feature values of first statement
# To use absolute value, uncomment first feature and comment second feature.

def one_to_one_dif(text, ref, startFColumn, feature_num):
    feature_list = []
    for i in range (startFColumn, feature_num):
        try:
            #feature = (abs(float(data2[i]) - float(data1[i])))
            feature = (float(text[i]) - float(ref[i]))
        except:
            feature = (float(text[i]))
                
        feature_list.append(round(feature, 2))
    return feature_list
    
    
def one_to_many_dif(ref, text, feature_num, feature_pos):
    avg_list = [];
    for i in range(len(ref)):
        feature_val = float(text[feature_pos]) - float(ref[i][feature_pos])
        avg_list.append(feature_val)
    avg = round(mean(avg_list), 2)
    return avg
    
    
# Input: news1 (String), news2 (String), outFile (String), startFColumn (integer)
# Description: Read the data from news1 (reliable) and news2 (unreliable), and put the data in two lists. Then, subtract the news1 values in each column from news2 values.
#              The subtraction starts from startFColumn until the end of the .csv file. Next, the results will be saved in the file with given name or path from user (outFile)
#
# Return: New DataFrame ('MS_features')

def multi_source_FE(news1, news2, outFile, startFColumn):
    
    # Reading data from .csv files
    data1 = data_read(f'./{news1}')
    data2 = data_read(f'./{news2}')
    
    # Saving original headers in header
    header = data1[1]
    feature_num = len(header)
    
    # Create header list with modified header names. Can be changed later with different method of multi-sourcing by modifying '- sub'.
    header_edit = ['text', 'news']
    header_edit = header_extension(header, 2, 'sub', header_edit)
    header_edit = header_extension(header, 2, 'avg sub', header_edit)
    
    #for i in range(startFColumn, len(data1[1])):
     #   header.append(f'{data1[1][i]} - sub')
      #  header.append(f'{data1[1][i]} - avg sub')
    
    # Assign data of the testing text and referencing text in data1 and data2
    data1 = data1[0]
    data2 = data2[0]
    
    # Create local lists, dataframe that are used later in this definition.
    text = []
    ref = []
    total_list = []
    new_total_list = []
    df = pd.DataFrame()
    
    # Loop through the data for subtraction
    for i in range(len(data2)):
        
        # Statement of each data (text - text trying to verify, ref - reliable articles to reference for text comparison)
        try:
            ref.append(data1[i][1])
            text.append(data2[i][1])
        except:
            pass
        
        #print(one_to_one_dif(data1[i], data2[i], startFColumn, feature_num))
        try:
            total_list.append(one_to_one_dif(data1[i], data2[i], startFColumn, feature_num))
            for j in range(startFColumn, feature_num):
                total_list[i].append(one_to_many_dif(data1, data2[i], feature_num, j))
        except:
            pass
        
    
    # Reshape the lists in terms of features. Ex. new_total_list[0] = all values of first feature
    for i in range(len(total_list)):
        new_list = []
        for element in (total_list):
            try:
                new_list.append(element[i])
            except:
                pass
        new_total_list.append(new_list)
       
    # Creating dataframe with multi-sourced features
    df['TEXT'] = text
    df['REFERENCE'] = ref
    for i in range(2, len(header_edit)):
        df[header_edit[i]] = new_total_list[i-2]
    
    # Saving data into file
    df.to_csv(f'./{outFile}')
    return df

# Initiate the code and check for the dataframe
MSFE = multi_source_FE('news_FE.csv', 'text_FE.csv', 'OtoM_FE.csv', 2)
print(MSFE.head(10))
